In [15]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
from utils import (process_vol,inspect_fiber_vol,filter_vol,
                   FiberGraph,get_high_density_mask,
                   generate_fake_fiber_data,inspect_vol)
from pathlib import Path
import os
#inspect_vol is also included in utils to avoid dependency on jlc
#from jlc import inspect_vol
"""
#Assumes the data is in the ./data/ folder as tif stacks
filenames = list(Path("./data/").glob("*.tif"))
filenames = sorted([str(f) for f in filenames])
print(filenames)
i = 0
vol = process_vol(filenames[i])
"""
#Synthetic data to avoid any file size problems
vol = generate_fake_fiber_data(H=128,num_fibers=16,num_frames=128,
                               bias_spatial=0.1,
                                bias_temporal=0.1,
                                bias_translation=0.05)
inspect_vol(vol)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#Inspect the vol (Use arrow keys to navigate through the different frames)
inspect_vol(vol)

In [7]:
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.imshow(vol[0],cmap='gray')
plt.title("First frame T=0"),plt.axis('off')
plt.subplot(132)
plt.imshow(vol[vol.shape[0]//2],cmap='gray')
plt.title(f"Middle frame, t={vol.shape[0]//2}"),plt.axis('off')
plt.subplot(133)
plt.imshow(vol[-1],cmap='gray')
plt.title(f"Last frame, t={vol.shape[0]-1}"),plt.axis('off')

(Text(0.5, 1.0, 'Last frame, t=255'), (-0.5, 127.5, 127.5, -0.5))

In [16]:
#define save_path to save the processed volume, avoid running this cell multiple times
save_path = None
#save_path = "./data/fake_fiber_data_processed.npy"
#default params
alpha = 0.01 #quantile for intensity normalization
thresh_small = [0.08,0.12] #threshold intervalfor volume after laplace filtering
s1,s2 = 0.75,1.5 #scales for the laplace (difference of gaussians) filter
vol_visualize_t = [0,vol.shape[0]//2,vol.shape[0]-1] #frames to visualize
vol_visualize = [[vol[j] for j in vol_visualize_t]]
vol, best_translation = process_vol(vol,
                                    reset_origo_on_bad_frame=False,
                                    crop_half=False,
                                    transpose=False,
                                    smaller=False)
vol_visualize.append([vol[j] for j in vol_visualize_t])
vol = filter_vol(vol,alpha=alpha,thresh_small=thresh_small)
vol_visualize.append([vol[j] for j in vol_visualize_t])
if save_path is not None:
    np.save(save_path,vol)

done with frame 0/127
done with frame 50/127
done with frame 100/127


In [9]:
plt.figure(figsize=(10,10))
concat_visualize = np.concatenate([np.concatenate(vol_visualize,axis=1) for vol_visualize in vol_visualize],axis=0)
plt.imshow(concat_visualize,cmap='gray')
h_vis,w_vis = vol_visualize[0][0].shape
xpos = range(w_vis//2,w_vis*len(vol_visualize[0]),w_vis)
ypos = range(h_vis//2,h_vis*len(vol_visualize),h_vis)
ax = plt.gca()
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
plt.xticks(xpos,[f"t={t}" for t in vol_visualize_t])
plt.yticks(ypos,["Original","Processed","Filtered"])

([<matplotlib.axis.YTick at 0x77fb4529aa60>,
 [Text(0, 64, 'Original'),
  Text(0, 192, 'Processed'),
  Text(0, 320, 'Filtered')])

In [10]:
#Inspect the vol (Use arrow keys to navigate through the different frames)
inspect_vol(vol)

In [17]:
fiber_graph = FiberGraph(radius=3,threshold=0.2)
fiber_graph.process_vol(vol)

#remove connected components with <min_size points
fiber_graph.cc_crop(min_size=3)

#remove leaf fibers with <=num points
fiber_graph.leaf_crop(num=2)

#remove high-density fiber areas (because they are too messy)
fiber_graph.high_dens_crop(get_high_density_mask(vol[-1]))

In [18]:
#show the last frame with fibers after removing various unwanted fibers
plt.imshow(vol[-1],cmap='gray')
fiber_graph.plot()

In [20]:
#inspect the volume with the fibers to show the temporal evolution
inspect_fiber_vol(vol,fiber_graph=fiber_graph)